In [1]:
import numpy as np
import os
import pickle


from bokeh.palettes import Colorblind8 as palette
import itertools  


from bokeh.sampledata.autompg import autompg
from bokeh.plotting import figure , output_file, save
from bokeh.io import output_notebook, show
from bokeh.layouts import column,gridplot

from bokeh.resources import CDN
from bokeh.embed import file_html

output_notebook()

Loading BokehJS ...

In [2]:
"""" 
/home/rs4070/optimizer_models/wideresnet/New_optimizer_plots/RK2_randomNumber

pickle.load(open(base+ ".p","rb"))
"""

def get_model_data2(optimizer, model_file) :
    filename_models = model_file

    pickle_files = os.listdir(filename_models)
    optimizer_models = [(p) for p in pickle_files if p.split("_")[0] == optimizer]
    
    optimizer_models = []
    for p in pickle_files:
        if len(optimizer.split("_"))>1:
            if p.split("_")[0] == optimizer.split("_")[0] and p.split("_")[1]==optimizer.split("_")[1]:
                optimizer_models.append(p)
            
        else:
            if p.split("_")[0] == optimizer and p.split("_")[1].isdigit() :
                optimizer_models.append(p)
        
        
    optimizer_files = [(os.listdir(filename_models + f)) for f in optimizer_models] 

    model_str = []
    for filename in optimizer_files:
        model_str.append([])
        for file_name in filename:
            if '.p' in file_name:
                model_str[-1].append(file_name)
                
    print(optimizer_models)
    
    nModels = len(model_str)   
    model_lr = [([(1./float(model_str[i][j].split('_')[1])) for j in range(len(model_str[i]))]) for i in range(nModels)]
    
    
    models = []
    for i in range(nModels):
#         models.append([])
        for j in range(len(model_str[i])):
            buf = {}
            buf['lr'] = model_lr[i][j]
            buf['model number'] = '_%s'%i
            buf['model data'] = pickle.load(open(filename_models + '/' + optimizer_models[i] + '/'  + model_str[i][j],"rb"))
            models.append(buf)
    
    print(optimizer + " learning rates = %s"%model_lr)

    return models

# a = get_model_data2('RK2', '/home/rs4070/optimizer_models/resnet_cifar/optimizer_plots/')
# print((a))

In [3]:
def plot_models_2(basefile,lr_dict):
    
    model_names = list(lr_dict.keys())
    models = []
    models = [(get_model_data2(name, basefile)) for name in model_names]
        
    "/home/rs4070/optimizer_models/linear_models_2/lasso_regression/optimizer_plots/"
    
    filename = "_"
    for name in model_names:
        filename += name + "_"

    output_file(basefile.split('/')[-3] + filename + ".html")

    options = dict(plot_width=400, plot_height=300,
               tools="pan,wheel_zoom,box_zoom,box_select,lasso_select")
        
    
    p = figure(width=800, height=600)
    p1 = figure(title="Test Loss - Log Plot",  **options)
    p2 = figure(title="Test Accuracy",  **options)
    p3 = figure(title="Training Loss - Log Plot",  **options)
    p4 = figure(title="Train Accuracy",  **options)
    
    colors = palette
       
    
    for j in range(len(model_names)):                   
        opti_models = [(m) for m in models[j] if m['lr'] in lr_dict[model_names[j]]]        
        opti_len = len(opti_models)
        
        for i,color in zip(range(opti_len),colors):
            if opti_models[i]['lr'] in lr_dict[model_names[j]]:                                                                          
                print(model_names[j] + opti_models[i]['model number'] ,opti_models[i]['model data']['args'])

                p1.line(np.arange(len(opti_models[i]['model data']['test loss'])),np.log(opti_models[i]['model data']['test loss']), color=colors[i + j],alpha=5.5, legend=model_names[j] + opti_models[i]['model number'] +' with lr = %s'%opti_models[i]['lr'])
                p2.line(np.arange(len(opti_models[i]['model data']['test accuracy'])),opti_models[i]['model data']['test accuracy'],color=colors[i + j], alpha=5.5, legend=model_names[j] + opti_models[i]['model number'] +' with lr = %s'%opti_models[i]['lr'])
                p3.line(np.arange(len(opti_models[i]['model data']['train loss'])),np.log(opti_models[i]['model data']['train loss']),color=colors[i +j], alpha=5.5, legend=model_names[j] + opti_models[i]['model number'] +' with lr = %s'%opti_models[i]['lr'])
                p4.line(np.arange(len(opti_models[i]['model data']['train accuracy'])),opti_models[i]['model data']['train accuracy'],color=colors[i +j], alpha=5.5, legend=model_names[j] + opti_models[i]['model number'] +' with lr = %s'%opti_models[i]['lr'])
                
                if 'train best perf' in opti_models[i]['model data']:
                    print(model_names[j] + opti_models[i]['model number'] +' with lr = %s'%opti_models[i]['lr'],'best test',opti_models[i]['model data']['test best perf']['accuracy'], 'best train=',opti_models[i]['model data']['train best perf']['accuracy'] )
                    print(model_names[j] + opti_models[i]['model number'] +' with lr = %s'%opti_models[i]['lr'],'best test',opti_models[i]['model data']['test best perf']['loss'], 'best train=',opti_models[i]['model data']['train best perf']['loss'] )
                    print('\n')
#                     p5.line(np.arange(len(opti_models[i]['model data']['train accuracy'])),opti_models[i]['model data']['train best perf']['accuracy'],color=colors[i +j], alpha=5.5, legend=model_names[j] + opti_models[i]['model number'] +' with lr = %s'%opti_models[i]['lr'])
#                     p6.line(np.arange(len(opti_models[i]['model data']['test accuracy'])),opti_models[i]['model data']['test best perf']['accuracy'],color=colors[i +j], alpha=5.5, legend=model_names[j] + opti_models[i]['model number'] +' with lr = %s'%opti_models[i]['lr'])



    p1.legend.location = 'top_right'
    p2.legend.location = 'bottom_right'
    p3.legend.location = 'top_right'
    p4.legend.location = 'bottom_right'
    p = gridplot([[ p3 ,p1] , [ p4, p2]], toolbar_location="right")    

#     p5.legend.location = 'top_right'
#     p6.legend.location = 'bottom_right'

    
    
#     if 'train best perf' in opti_models[-1]['model data']:
#         p = gridplot([[ p3 ,p1] , [ p4, p2], [p5,p6]], toolbar_location="right")  
#         p5.legend.location = 'top_right'
#         p6.legend.location = 'bottom_right'

#     else:
#         p = gridplot([[ p3 ,p1] , [ p4, p2]], toolbar_location="right")  
        
    show(p)
    



In [4]:
# lr_dict = {'SGD':[0.1], 'RK2':[0.5,0.1]}

# plot_models_2('/home/rs4070/optimizer_models/wideresnet/optimizer_plots/', lr_dict)


In [5]:
# lr_dict = {'SGD':[0.1], 'RK2':[0.1]}
# lr_dict = {'SGD':[0.5], 'RK2':[0.5]}
# lr_dict = {'SGD':[0.01], 'RK2':[0.01]}
# lr_dict = {'SGD':[0.05], 'RK2':[0.05]}

# lr_dict = {'RK2':[0.05],'SGD':[0.05], 'RK4':[0.1]}

# lr_dict = {'RK2':[0.1], 'SGD':[0.1], 'RK4':[0.1]}
# lr_dict = {'RK2':[0.01], 'SGD':[0.01], 'RK4':[0.01]}
# lr_dict = {'RK2':[0.05], 'SGD':[0.05], 'RK4':[0.05]}


# plot_models_2('/home/rs4070/optimizer_models/linear_models/lasso_regression/optimizer_plots/', lr_dict)

In [29]:
# lr_dict = {'SGD':[0.1], 'RK2':[0.1]}
# lr_dict = {'SGD':[0.5], 'RK2':[0.5]}
# lr_dict = {'SGD':[0.01], 'RK2':[0.01]}
# lr_dict = {'SGD':[0.05], 'RK2':[0.01]}

lr_dict = {'SGD':[0.5], 'RK2':[0.1]}
# lr_dict = {'RK2':[-0.1, 0.01,-0.5, 0.05]}

# lr_dict = {'RK2':[0.01], 'SGD':[0.01], 'RK4':[0.01]}
# lr_dict = {'RK2':[0.01], 'SGD':[0.01], 'RK4':[0.01]}
# lr_dict = {'RK2':[0.05], 'SGD':[0.05], 'RK4':[0.05]}


plot_models_2('/home/rs4070/optimizer_models/linear_models/logistic_regression/optimizer_plots_2/', lr_dict)

FileNotFoundError: [Errno 2] No such file or directory: '/home/rs4070/optimizer_models/linear_models/logistic_regression/optimizer_plots_2/'

In [7]:
# lr_dict = {'RK2':[0.1], 'SGD_momentum':[0.1], 'SGD':[0.1]}

# lr_dict = {'RK2':[0.5], 'SGD_momentum':[0.1], 'SGD':[0.1]}


# plot_models_2('/home/rs4070/optimizer_models/resnet_cifar/optimizer_plots_1/', lr_dict)


In [30]:
lr_dict = {'RK2':[0.05, 0.1], 'SGD':[0.01]}
# lr_dict = {'RK2':[-0.05 ,0.01], 'Adagrad':[0.1]}


plot_models_2('/home/rs4070/optimizer_models/linear_models_2/lasso_regression/optimizer_plots/', lr_dict)


['RK2_63968_32375', 'RK2_4868_93185', 'RK2_93464_80614']
RK2 learning rates = [[0.01], [0.1], [0.05]]
['SGD_85935_97008', 'SGD_83789_56811', 'SGD_74632_56840']
SGD learning rates = [[0.05], [0.1], [0.01]]
RK2_1 Namespace(batch_size=128, dataset='mnist', epoch_step='[-10,-50]', epochs=50, lr=0.1, lr_decay=0.1, model='Logistic_Regression', momentum=0.9, nesterov=True, optimizer='RK2', wd=0.005)
RK2_1 with lr = 0.1 best test 92.47 best train= 100.0
RK2_1 with lr = 0.1 best test 0.002152528512850404 best train= 0.05341758206486702


RK2_2 Namespace(batch_size=128, dataset='mnist', epoch_step='[-10,-50]', epochs=50, lr=0.05, lr_decay=0.1, model='Logistic_Regression', momentum=0.9, nesterov=True, optimizer='RK2', wd=0.005)
RK2_2 with lr = 0.05 best test 92.65 best train= 97.65625
RK2_2 with lr = 0.05 best test 0.0021030272662639618 best train= 0.07883455604314804


SGD_2 Namespace(batch_size=128, dataset='mnist', epoch_step='[-10,-50]', epochs=50, lr=0.01, lr_decay=0.1, model='Logistic_Regre

In [9]:
# lr_dict = {'RK2':[0.1], 'RK2_heun':[0.1], 'SGD_momentum':[0.1], 'SGD':[0.1]}

# lr_dict = {'RK2':[0.5], 'SGD_momentum':[0.1], 'SGD':[0.1]}


# plot_models_2('/home/rs4070/optimizer_models/resnet_cifar/optimizer_plots_2/', lr_dict)

In [10]:
# lr_dict = {'RK2':[0.1, -0.5, 0.05],'RK2_heun':[0.1],'SGD_momentum':[0.1], 'SGD':[0.1]}

lr_dict = {'RK2':[0.1,0.05], 'RK2_heun':[-0.1], 'SGD_momentum':[0.1], 'SGD':[-0.1]}

# lr_dict = {'RK2':[0.1, -0.5, 0.05],'RK2_heun':[0.1], 'SGD':[0.1]}

plot_models_2('/home/rs4070/optimizer_models/resnet_cifar/optimizer_plots/', lr_dict)

['RK2_13372_26589', 'RK2_71651_51635', 'RK2_46587_74807', 'RK2_84098_65416']
RK2 learning rates = [[0.5], [0.5], [0.05], [0.1]]
['RK2_heun_96779_76622', 'RK2_heun_2274_12265', 'RK2_heun_37413_49840']
RK2_heun learning rates = [[0.5], [0.5], [0.1]]
['SGD_momentum_47198_37798', 'SGD_momentum_65080_86881', 'SGD_momentum_68838_46485']
SGD_momentum learning rates = [[0.1], [0.1], [0.1]]
['SGD_24887_43259']
SGD learning rates = [[0.1]]
RK2_2 Namespace(epoch_step='[-10,-50]', epochs=200, lr=0.05, lr_decay=0.1, momentum=0.9, nesterov=True, optimizer='RK2', resume=False, wd=0.0005)
RK2_2 with lr = 0.05 best test 92.44 best train= 100.0
RK2_2 with lr = 0.05 best test 0.19850383698940277 best train= 5.529075860977173e-05


RK2_3 Namespace(epoch_step='[20,50,80]', epochs=200, lr=0.1, lr_decay=0.1, momentum=0.9, nesterov=True, optimizer='RK2', resume=False, wd=0.0005)
RK2_3 with lr = 0.1 best test 92.5 best train= 100.0
RK2_3 with lr = 0.1 best test 0.2129983901977539 best train= 0.0001900121569633

In [24]:
# lr_dict = {'RK2':[0.1, -0.5, 0.05],'RK2_heun':[0.1],'SGD_momentum':[0.1], 'SGD':[0.1]}

lr_dict = {'RK2':[0.05, 0.51,0.5, 0.1],'SGD':[0.1,-0.5]}

# lr_dict = {'RK2':[0.1, -0.5, 0.05],'RK2_heun':[0.1], 'SGD':[0.1]}

plot_models_2('/home/rs4070/optimizer_models/wideResnet_cifar/optimizer_plots/', lr_dict)

['RK2_52593_82339', 'RK2_99714_67795', 'RK2_61799_2200', 'RK2_66854_87198', 'RK2_6548_94962']
RK2 learning rates = [[0.05], [0.5], [], [0.1], [0.51]]
['SGD_23386_5540', 'SGD_17377_20686', 'SGD_73429_41537']
SGD learning rates = [[0.1], [0.1], [0.5]]
RK2_0 Namespace(batchSize=128, cuda=True, dataroot='.', dataset='CIFAR10', depth=16, dtype='float', epoch_step='[-30,-50]', epochs=200, gpu_id='0', groups=1, lr=0.05, lr_decay_ratio=0.1, model='resnet', momentum=0.9, ngpu=1, nthread=4, optimizer='RK2', randomcrop_pad=4, resume='', save='./logs/resnet_167221281', weightDecay=0.0005, width=10.0)
RK2_0 with lr = 0.05 best test 92.47 best train= 99.95266666666667
RK2_0 with lr = 0.05 best test 0.3714169587798511 best train= 0.002109442670034238


RK2_1 Namespace(batchSize=128, cuda=True, dataroot='.', dataset='CIFAR10', depth=16, dtype='float', epoch_step='[80]', epochs=200, gpu_id='0', groups=1, lr=0.5, lr_decay_ratio=0.1, model='resnet', momentum=0.9, ngpu=2, nthread=4, optimizer='RK2', rando